In [1]:
cd C:\Users\ab42313\OneDrive\ABZ_PhD\Programming\NCBI

In [2]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna

handle = open("genBankRecord_545778205.fasta", "rU")
fasta = list(SeqIO.parse(handle, "fasta", generic_dna))

seqs = []
for i in range(0,len(fasta)):
    seqs.append(fasta[i].seq)
    #print fasta[i].seq

genome = seqs[0]

#MiSeq primers (5'-3')
#fw = 'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG'
#rv = 'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG'

#Primers from wiki, 8F and 1492R
fw = 'AGAGTTTGATCCTGGCTCAG'
rv = 'CGGTTACCTTGTTACGACTT'

In [3]:
def simulate_pcr_run(genome, fw_primer, rv_primer, approximative_gene_lenght = 2600, mismatches = 2):
    '''
    -approximative_gene_lenght: the approximative expected lenght of the gene
    -mismatches: is set on 2 as in an highly stringent PCR for MiSeq sequencing
    '''

    
    
    ##Humming distance calculator is used to mimik primer/probe mismatches as PCR stringency degree
    
    def calculate_hamming_distance(seq1,seq2):
        hamming = 0
        if len(seq1) > len(seq2):
            hamming += len(seq1) - len(seq2)
            for i in range(0, len(seq2)):
                if seq1[i] != seq2[i]:
                    hamming += 1
            return hamming
        elif len(seq1) < len(seq2):
            hamming += len(seq2) - len(seq1)
            for i in range(0, len(seq1)):
                if seq1[i] != seq2[i]:
                    hamming += 1
            return hamming
        else:
            for i in range(0, len(seq1)):
                if seq1[i] != seq2[i]:
                    hamming += 1
            return hamming
    
    
    
    
    #primer_positions = []
    sequences = []
    gene_len = approximative_gene_lenght + 100
    fw = fw_primer
    rv = rv_primer

    positions = range(0,len(genome)-len(fw))
    len(positions)

    
    
    
    for i in positions:
        
        testing = genome[i:i+len(fw)]
        
        if calculate_hamming_distance(testing, fw) <= mismatches:
            
            for m in range(i + len(rv), gene_len):
                
                if calculate_hamming_distance(genome[m : m + len(rv)], rv) <= mismatches:
                    
                    if len(genome[i:m+len(rv)]) > 10:
                        
                        #print i, m, genome[i:m]
                        sequences.append(str(genome[i:m+len(rv)]))
                        #primer_positions.append(i)
                        #primer_positions.append(m+len(rv))
                  
                        #break
    
    return sequences
    #return 'Positions: ', primer_positions
    #return sequences

In [6]:
sequences = simulate_pcr_run(genome, fw, rv, approximative_gene_lenght =1500, mismatches = 10) 
for i in sequences:
    print i

...


In [7]:
proper_lenght_seqs = []
for i in sequences:
    if 1300 < len(i) < 1500:
        proper_lenght_seqs.append(i)
print proper_lenght_seqs

['AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAATTAAAATTTTATTGACTTAGGTCACTAAATACTTTAACCAATATAGGCATAGCGCACAGACAGATAAAAATTACAGAGTACACAACATCCATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGACGCGTACAGGAAACACAGAAAAAAGCCCGCACCTGACAGTGCGGGCTTTTTTTTTCGACCAAAGGTAACGAGGTAACAACCATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAACGTTTTCTGCGTGTTGCCGATATTCTGGAAAGCAATGCCAGGCAGGGGCAGGTGGCCACCGTCCTCTCTGCCCCCGCCAAAATCACCAACCACCTGGTGGCGATGATTGAAAAAACCATTAGCGGCCAGGATGCTTTACCCAATATCAGCGATGCCGAACGTATTTTTGCCGAACTTTTGACGGGACTCGCCGCCGCCCAGCCGGGGTTCCCGCTGGCGCAATTGAAAACTTTCGTCGATCAGGAATTTGCCCAAATAAAACATGTCCTGCATGGCATTAGTTTGTTGGGGCAGTGCCCGGATAGCATCAACGCTGCGCTGATTTGCCGTGGCGAGAAAATGTCGATCGCCATTATGGCCGGCGTATTAGAAGCGCGCGGTCACAACGTTACTGTTATCGATCCGGTCGAAAAACTGCTGGCAGTGGGGCATTACCTCGAATCTACCGTCGATATTGCTGAGTCCACCCGCCGTATTGCGGCAAGCCGCATTCCGGCTGATCACATGGTGCTGATGGCAGGTTTCACCGCCGGTAATGAAAAAGGCGAACTGGTGGTGCTTGGACGCAACGGTTCCGACTACTCTGCTGCGGTGCTGGCTGCCTGTTTACGCGCCGA

The program apparently works since it selects fragments.

Problems:
    - "amplified" fragments blast result is not informative at all (raises an error...)
    - MiSeq primers don't work.
    - Primer/Probe mismatches have to be set at high values
    
Will check the code again on monday.